<a href="https://colab.research.google.com/github/hansschaa/Machine_Learning_Exercises/blob/master/RNN/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data

In [32]:
import os

# Ruta de la carpeta
folder = 'training_data/'
seqLenght = 200
memory = 2
epochs = 200

# Lista de archivos en la carpeta
files = [f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f)) and f.endswith('.txt')]

charsList = ['$', 'B','E','>',']', 'S', 'o', '<', '[', 'b', '?', 'Q']
# Crear una tabla de traducción
translateTable = str.maketrans({char: '-' for char in charsList})

# Extraer columnas del nivel filtradas
def extractColumns(level):
    columns = []
    for i in range(len(level[0])):
        column = ''.join(fila[i] for fila in level)
        columnFiltered = column.translate(translateTable)
        columns.append(columnFiltered)
    return columns

levelColumns = []

# Itera sobre cada archivo
for file in files:
    with open(os.path.join(folder, file), 'r') as f:
        # Lee el contenido del archivo y divide el nivel por líneas
        level = f.read().strip().split('\n')
        # Llamamos a la función y guardamos las columnas en una variable
        levelColumns += extractColumns(level)

#conjunto sin duplicados
states = set(levelColumns)
states = list(states)

In [33]:
import random

# Diccionario con los estados mapeados a un índice
state_index = {state: idx for idx, state in enumerate(states)}

# Diccionario con los indices mapeados a un estado
index_states = {idx: state for state, idx in state_index.items()}

X = []

# Asignar índices a las columnas
for column in levelColumns:
    index = state_index[column]
    X.append(index)

Y = X[:]

# Desplazar la lista hacia la izquierda
'''Y = Y[memory:]'''

# Agregar tantos valores aleatorios al final de Y como sea necesario para que X e Y tengan la misma dimensión
'''for _ in range(memory):
    randomValue = random.choice(Y)
    Y.append(randomValue)'''

print(X)
print(Y)

[6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 28, 28, 6, 6, 28, 28, 6, 28, 28, 28, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 28, 28, 0, 0, 0, 28, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 50, 26, 13, 28, 6, 6, 28, 28, 26, 26, 13, 28, 33, 33, 33, 28, 6, 6, 6, 6, 28, 2, 2, 2, 28, 28, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 28, 28, 6, 6, 6, 6, 6, 6, 40, 50, 26, 13, 20, 20, 20, 28, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 40, 50, 26, 29, 29, 6, 6, 6, 6, 6, 28, 28, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 28, 28, 3, 3, 3, 3, 43, 17, 17, 17, 17, 17, 0, 0, 6, 21, 21, 10, 28, 28, 4, 4, 4, 4, 4, 4, 4, 33, 33, 33, 22, 49, 49, 49, 49, 49, 49, 25, 25, 12, 37, 37, 37, 28, 2, 2, 2, 2, 28, 23, 23, 23, 6, 23, 23, 23, 6, 23, 23, 23, 6, 15, 15, 15, 6, 2, 2, 2, 47, 25, 25, 25, 10, 10, 10, 10, 28, 33, 33, 24, 39, 24, 33, 33, 28, 25, 25, 25, 39, 39, 39, 16, 49, 49, 49, 25, 28

##RNN


In [34]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense

# Define cuántos pasos temporales se considerarán
time_steps = memory
# Define el número de características en cada paso temporal
num_features = 1


X_train = np.array(X)
Y_train = np.array(Y)

# Reformatear para que sea adecuado para una RNN
'''X = [
    [1, 2, 3],
    [2, 3, 4],
    [3, 4, 5],
    [4, 5, 6],
    [5, 6, 7],
    [6, 7, 8],
    [7, 8, 9],
    [8, 9, 10]
]'''
X_train = [X[i:i+time_steps] for i in range(len(X)-time_steps)]

'''Y_train = [4, 5, 6, 7, 8, 9, 10]'''
Y_train = Y[memory:]


# Convertir a formato tridimensional
'''X_train = np.array([
    [[1], [2], [3]],
    [[2], [3], [4]],
    [[3], [4], [5]],
    [[4], [5], [6]],
    [[5], [6], [7]],
    [[6], [7], [8]],
    [[7], [8], [9]],
    [[8], [9], [10]]
])'''
X_train = np.array(X_train).reshape(-1, time_steps, num_features)
Y_train = np.array(Y_train)


# Generar conjunto de datos para entrenar y testear
X_train, X_test, Y_train, Y_test = train_test_split(X_train, Y_train, test_size=0.2, random_state=6)

# Arquitectura
# Soft max, útil para clasificación (0.8, 0.1, 0.1)
model = Sequential([
    SimpleRNN(units=52, activation='relu', input_shape=(memory, 1)),
    Dense(units=100, activation='relu'),
    Dense(units=100, activation='relu'),
    Dense(units=52, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

for i in range(0,epochs):
  model.fit(X_train, Y_train, epochs = 1, batch_size=12)
  model.evaluate(X_test, Y_test)

#

# Hacer predicción
predicciones = model.predict(X_test)



'''for i, secuencia in enumerate(predicciones[:30]):
    print(f"Secuencia {i+1}: {secuencia}")'''

# Suponiendo que 'predicciones' es la matriz de probabilidades que obtuviste
clases_predichas = np.argmax(predicciones, axis=1)

# Imprimir cada elemento de clases_predichas hacia el lado
for clase in clases_predichas:
    print(clase, end=' ')

19/19 [==============================] - 0s 1ms/step
25 6 6 28 6 6 2 6 6 6 6 6 6 6 6 33 6 6 6 6 6 6 28 10 6 6 6 6 6 33 6 6 23 28 6 6 6 6 28 6 6 6 35 6 6 14 10 6 6 39 6 6 6 6 6 6 6 6 28 6 6 6 28 6 6 40 51 6 28 6 6 28 6 6 17 6 6 33 6 4 6 6 6 6 6 28 6 28 6 6 23 17 17 6 6 6 6 2 6 28 26 6 28 6 6 28 6 28 6 50 6 10 6 6 50 6 6 6 6 6 6 6 49 33 28 29 6 6 6 6 6 28 6 6 28 6 41 10 6 29 6 6 6 17 6 50 6 6 6 6 6 26 2 0 26 6 10 6 6 6 10 6 6 6 28 6 6 6 28 6 6 6 39 6 28 6 6 2 6 6 6 6 39 28 28 6 6 26 6 6 6 6 6 50 6 6 6 6 10 6 6 6 6 6 6 6 39 6 6 6 6 6 49 6 6 6 6 6 6 39 29 6 50 33 6 33 6 6 6 28 6 26 33 28 6 6 6 29 6 6 50 6 6 2 6 6 6 6 6 18 25 10 6 6 6 6 6 6 6 28 6 6 6 6 50 6 6 6 28 4 6 6 6 39 51 6 6 6 6 28 6 41 0 6 6 6 28 6 28 6 6 6 6 29 6 6 33 6 6 3 6 6 6 28 6 6 26 6 6 50 4 6 6 26 28 28 6 6 6 28 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 28 6 6 6 6 6 10 28 6 4 6 6 6 28 6 40 6 6 6 17 6 50 6 28 6 0 28 6 6 6 6 6 6 51 6 41 6 6 6 28 6 6 51 6 6 6 6 26 6 3 6 18 6 6 6 6 6 6 6 6 6 6 6 14 6 6 6 6 50 6 6 6 6 6 6 6 6 28 28 6 33 28

In [35]:
# Convertir los índices predichos a estados usando el diccionario
sequence = [index_states[idx] for idx in clases_predichas]

def transpose_sequence(sequence):
    heights = []
    level = ""

    colCount = 0
    charCount = 0
    maxCol = len(sequence)
    maxChar = len(sequence[0])
    level = ""

    while charCount < maxChar:
        while colCount < maxCol:
            level += sequence[colCount][charCount]
            colCount += 1
        charCount += 1
        colCount = 0
        level += '\n'

    return level

resultado = transpose_sequence(sequence)
print(resultado)

print("*********************************")


# Convertir los índices predichos a estados usando el diccionario
sequence = [index_states[idx] for idx in Y_test]

resultado = transpose_sequence(sequence)
print(resultado)

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------